<a href="https://colab.research.google.com/github/nerudxlf/omstu_reference_groups/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from google.colab import files

files.upload()

In [ ]:
def get_normal_value(data: list) -> list:
  """
  Функция удаляет из строк в списке все символы кроме букв пробела и ;
  param: data(list): список категорий из выгрузки WoS
  return: return_list(list): преобразованный список категорий
  """
  return_list = []
  for item in data:
    item = re.sub(r'[^A-Za-z ;]', '', item)
    return_list.append(item)
  return return_list


def count_values(wos_categories: list, name_code: dict) -> DataFrame:
  """
  Функция считает количество статей по направлениям
  param: wos_categories(list): категории статей из выгрузки WoS
         name_code(dict): словарь, где ключ - это название референтной группы
                                       значение - это код классификации отраслей
  return: DataFrame: таблица из названия референтной группы и количества
                     публикаций, которые входят в эту группу
  """
  name_value_dict = dict.fromkeys(name_code.keys(), 0)
  for key, value in name_code.items():
    for element in wos_categories:
      for category in element.split("; "):
        try:
          if value.find(category.upper()) != -1:
            name_value_dict[key] += 1
            break
        except AttributeError:
          pass
  return pd.DataFrame({"Name": name_value_dict.keys(), "Value": name_value_dict.values()})

In [ ]:
all_wos_2020_df = pd.read_excel("data/all_wos_2020.xls")
schema_df = pd.read_excel("data/schema.xlsx")

name_schema_list = schema_df["name"].to_list()
code_schema_list = schema_df["code"].to_list()

name_code_dict = dict(zip(name_schema_list, code_schema_list))
wos_categories_tuple = get_normal_value(all_wos_2020_df["WoS Categories"].to_list())
result_df = count_values(wos_categories_tuple, name_code_dict)
result_df.to_excel("data/results.xlsx", index=False)